# Libraries

In [1]:
# Libraries
import pokebase as pb
import requests
import json
import pandas as pd
import numpy as np

# Collect the Data

In [2]:
query = requests.get('https://pokeapi.co/api/v2/move?limit=1000&offset=0')
query_data = json.loads(query.text)
print(query_data.keys())

dict_keys(['count', 'next', 'previous', 'results'])


In [3]:
print(len(query_data["results"]))

844


In [4]:
move_data = query_data["results"]

# Clean the Data

In [5]:
# moves_df

In [6]:
# This cell is for data exploration
test = move_data[53]
# print(test.keys())
print(test["name"])

move_query = requests.get("https://pokeapi.co/api/v2/move/ember")
move_query_data = json.loads(move_query.text)
print(move_query_data.keys())

mist
dict_keys(['accuracy', 'contest_combos', 'contest_effect', 'contest_type', 'damage_class', 'effect_chance', 'effect_changes', 'effect_entries', 'flavor_text_entries', 'generation', 'id', 'learned_by_pokemon', 'machines', 'meta', 'name', 'names', 'past_values', 'power', 'pp', 'priority', 'stat_changes', 'super_contest_effect', 'target', 'type'])


In [7]:
move_query_data["effect_entries"]

[{'effect': 'Inflicts regular damage.  Has a $effect_chance% chance to burn the target.',
  'language': {'name': 'en', 'url': 'https://pokeapi.co/api/v2/language/9/'},
  'short_effect': 'Has a $effect_chance% chance to burn the target.'}]

In [8]:
move_query_data["effect_changes"]

[]

In [9]:
move_query_data["generation"]

{'name': 'generation-i', 'url': 'https://pokeapi.co/api/v2/generation/1/'}

**NOTES**
- need to heck the `past_values` key to makes ure that the data we are getting for those things is up to date with the infomration we want
- need to substitue `effect_chance` info `effect_entries`

In [10]:
colums_needed = ["name", "accuracy", "class", "game_desc", "effect_chance", "generation", "machines", "crit_rate", "type", "power", "pp", "priority"]
gens = ['generation-i', 'generation-ii', 'generation-iii', 'generation-iv', 'generation-v']
latest_game = "back-2-white-2"
second_latest_game = "black-white"
moves_df = pd.DataFrame(columns=colums_needed)

In [11]:
for move in move_data:
    # pull the page data that we need
    move_query = requests.get(move["url"])
    move_query_data = json.loads(move_query.text)
    # print(move_query_data["generation"])
    if move_query_data["generation"]["name"] in gens: # only get the abilities from the first 5 generation
        # print("got here")
        # print("here")
        temp = [""] * len(colums_needed)

        temp[0] = move_query_data["name"] # get the move's name

        temp[1] = move_query_data["accuracy"] # get the accuracy of the move
        
        temp[2] = move_query_data["damage_class"]["name"] # get the damgage type of the class
        
        effect_data = move_query_data["effect_entries"] # get the in game description of the move
        for text in effect_data:
            if text["language"]["name"] == "en": # get the english descriptions from black and white
                desc = text["short_effect"]
                desc = desc.replace('$effect_chance', str(move_query_data["effect_chance"]))
                temp[3] = desc
                desc = text["effect"].replace('$effect_chance', str(move_query_data["effect_chance"]))
                print(desc)
                temp[4] = desc
                break
                    
        moves_df.loc[len(moves_df.index)] = temp
                    
        # get the generation that the ability is from
#         gen = ability_query_data["generation"]["name"]

#         if "iv" in gen:
#             temp[1] = 4
#         elif "v" in gen:
#             temp[1] = 5
#         elif "iii" in gen:
#             temp[1] = 3
#         elif "ii" in gen:
#             temp[1] = 2
#         elif "i" in gen:
#             temp[1] = 1
#         else: # raise an error if we get a value that is not one of these generations
#             print(gen)
#             raise ValueError("We got an invalid generation!")

#         effect_data = ability_query_data["effect_entries"] # get the effect description
#         for text in effect_data:
#             if text["language"]["name"] == "en": # get the english descriptions
#                     temp[2] = text["effect"]
#                     break

        

        # print(temp)

        # moves_df.loc[len(moves_df.index)] = temp                

Inflicts regular damage.
Inflicts regular damage.  User's critical hit rate is one level higher when using this move.
Inflicts regular damage.  Hits 2–5 times in one turn.

Has a 3/8 chance each to hit 2 or 3 times, and a 1/8 chance each to hit 4 or 5 times.  Averages to 3 hits per use.
Inflicts regular damage.  Hits 2–5 times in one turn.

Has a 3/8 chance each to hit 2 or 3 times, and a 1/8 chance each to hit 4 or 5 times.  Averages to 3 hits per use.
Inflicts regular damage.
Inflicts regular damage.  After the battle ends, the winner receives five times the user's level in extra money for each time this move was used.
Inflicts regular damage.  Has a 10% chance to burn the target.
Inflicts regular damage.  Has a 10% chance to freeze the target.
Inflicts regular damage.  Has a 10% chance to paralyze the target.
Inflicts regular damage.
Inflicts regular damage.
Inflicts damage equal to the target's max HP.  Ignores accuracy and evasion modifiers.  This move's accuracy is 30% plus 1% fo

In [12]:
moves_df

,name,accuracy,class,game_desc,effect_chance,generation,machines,crit_rate,type,power,pp,priority
0,pound,100,physical,Inflicts regular damage with no additional eff...,Inflicts regular damage.,,,,,,,
1,karate-chop,100,physical,Has an increased chance for a critical hit.,Inflicts regular damage. User's critical hit ...,,,,,,,
2,double-slap,85,physical,Hits 2-5 times in one turn.,Inflicts regular damage. Hits 2–5 times in on...,,,,,,,
3,comet-punch,85,physical,Hits 2-5 times in one turn.,Inflicts regular damage. Hits 2–5 times in on...,,,,,,,
4,mega-punch,85,physical,Inflicts regular damage with no additional eff...,Inflicts regular damage.,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
572,shadow-hold,None,status,Prevents the target from leaving battle.,The target cannot switch out normally. Ignore...,,,,,,,
573,shadow-mist,100,status,Lowers the target's evasion by two stages.,Lowers the target's evasion by two stages.,,,,,,,
574,shadow-panic,90,status,Confuses the target.,Confuses the target.,,,,,,,
575,shadow-shed,None,status,"Removes Light Screen, Reflect, and Safeguard.","Removes the effects of light screen, reflect, ...",,,,,,,


In [13]:
moves_df.loc[0, "game_desc"]

'Inflicts regular damage with no additional effect.'